In [1]:
import re, string, nltk
import numpy as np
import tensorflow as tf
import tensorflow_hub as tf_hub
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords, wordnet
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

%pip install -U -q ibm-watson-machine-learning

api_key='vD5MYd1qlPmuSio9ubmytdXmbUApziO74nS969CDjtAo'
location = 'us-south'

wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

client.spaces.list(limit=5)

space_id = '9b602fc8-d468-41cb-a19a-b140f78e4d00' #id dari space yg telah dibuatt

client.set.default_space(space_id)

new_text = "Balanced Labs exists to improve the lives of accountants and their clients with intelligent and elegant tools. Weâ€™re an energetic team of ex-Xeroâ€™s, COOs, designers, and engineers based in San Francisco and Sydney.Â As our Communication Designer, you will help define our brand experience and how we communicate with our community.In this role, you will be responsible for our brand experience, and youâ€™ll work across our product and marketing design teams to define how our platform connects and resonates with our customers.You have a deep passion for communication design, and are willing to share your expertise with others.Balanced Labs is an Equal Opportunity Employer."

max_features = 10000
# Membuat Tokenizer
t = Tokenizer(num_words = max_features)
# fit tokenizer
t.fit_on_texts(new_text)

stop_words = set(stopwords.words("english"))
punctuation = list(string.punctuation)
stop_words.update(punctuation)

lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def text_preprocessing(text):
  # Case folding
  text = text.lower()

  # Newline removal (\n)
  text = re.sub(r"\\n", " ",text)

  # Whitespace removal
  text = text.strip()

  # URL removal
  text = re.sub(r"http\S+", " ", text)
  text = re.sub(r"www.\S+", " ", text)

  # Tokenization
  tokens = word_tokenize(text)

  # Stopwords removal
  tokens = [word for word in tokens if word not in stop_words]

  # Lemmatization
  tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]

  # Combining Tokens
  text = ' '.join(tokens)

  return text

text_prepoc = text_preprocessing(new_text)

encoded_docs_testing = t.texts_to_sequences(text_prepoc)

encoded_docs_testing = t.texts_to_sequences(text_prepoc)

embedded_docs_testing = pad_sequences(encoded_docs_testing,padding='pre',maxlen=100000)

testing_data = np.array(embedded_docs_testing)

scoring_payload = {"input_data": [{"values": testing_data}]}
deployment_id= '91213a22-e99b-4644-bf07-9501dbf54991'

predictions = client.deployments.score(deployment_id, scoring_payload)

scoring_response_json = predictions
print(scoring_response_json)

# Extracting and printing the prediction results
prediction = scoring_response_json['predictions'][0]
predicted_probabilities = prediction['values'][0][0]
predicted_class = prediction['values'][0][1]

print(f"Predicted class: {predicted_class}")
print(f"Predicted probabilities: {predicted_probabilities}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Note: you may need to restart the kernel to use updated packages.
------------------------------------  ----------------------------------------  ------------------------
ID                                    NAME                                      CREATED
9b602fc8-d468-41cb-a19a-b140f78e4d00  Deployment Kelompok 2 - Capstone Project  2024-06-15T09:38:11.836Z
6d8cddd1-c3b8-4c45-979d-5c8b81cd106b  Capstone                                  2024-06-14T09:50:19.401Z
889c1e3c-484a-4a1a-8f4b-3de337b23f6d  Deploy                                    2024-06-11T00:42:01.970Z
------------------------------------  ----------------------------------------  ------------------------
